In [2]:
from numpy import pi, cos, sin, sqrt
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import pandas as pd
from math import modf
import json
from random import uniform
import requests
from numpy import mean

hex_width = 50
city_radius = 0.20 # in lat-long degrees

hex_size = city_radius/(2*hex_width)

washington_y = 38.904978
washington_x = -77.039658


atlanta_y = 33.7490
atlanta_x = -84.3880


In [3]:
# df = gpd.GeoDataFrame(pd.read_pickle('../transit/points_pickle'),crs={'init' :'epsg:3857'})

In [4]:
def axial_to_cube(x,y):
    z = -x -y
    return (x,y,z)

In [5]:
key_path = "C:\\Users\\John\\Documents\\notGitHub\\API_key.txt"
key_file = open(key_path, 'r')
api_key = key_file.readline()

In [6]:
string1 = "https://mobilityscore.transitscreen.io/api"
string2 = "/v1/locations.json?coordinates="
string3 = "&key="
string4 = "&geojson=true"

def api_string(coords):
    new_string = string1+string2+coords+string3+api_key+string4
    return new_string

In [7]:
class hexagon:
    def __init__(self, coords, city_center=(atlanta_y, atlanta_x), size=hex_size):
        # there will be no coordinate reference system at the 
        # hex level, that needs to be handled by the grid
        self.x_loc = float(coords[0])
        self.y_loc = float(coords[1])
        self.z_loc = float(coords[2])
        self.city_center = city_center
        self.size = size
        self.self_center()
        self.create_vertices()
        self.points=[]
        self.scores=[]
        self.bikeshares=[]
        self.carshares=[]
        self.masstransits=[]
        self.ridehailings=[]
    def self_center(self):
        self.x_center = self.size * 1.5 * self.x_loc
        self.y_center = self.size * sqrt(3) * (self.y_loc + self.x_loc/2)        
    def create_vertices(self):
        # I am only interested in making "flat top" layouts
        self.vertices = tuple(self.vertex(n) for n in range(6))
    def vertex(self, n):
        angle = n*pi/3
        i = self.size*cos(angle) + self.x_center + self.city_center[1]
        j = self.size*sin(angle) + self.y_center + self.city_center[0]
        return(i,j)
        
    
    def random_points(self, n):
        for i in n:
            rand_x = uniform(-1,1)
            rand_y = uniform(-1,1)
            point_h = self.center_x + rand_x *self.size
            point_v = self.center_y + self.size*sqrt(3) * (rand_y + rand_x / 2)
            
            self.points.append((point_v,point_h))
            # note that this gives the point in lat-long format which means y comes first!
            
    def coord_string(pair):
        # note that this assumes the points are already in lat-long format!
        return str(pair[0]) + "," + str(pair[1])
    
    def api_scrape(self):
        for coord in self.points:
            request_string = api_string(coord_string(pair))
            result = requests.get(request_string)
            # this needs to use raw score breakdown
            if result.status_code == 200:
                self.scores.append(result.json()['data']['mobilityScore']['score'])
                breakdown = result.json()['data']['mobilityScore']['scoreBreakdown']
                self.bikeshares.append(float(breakdown['bikeshare'].strip('%'))/100.0)
                self.carshares.append(float(breakdown['carshare'].strip('%'))/100.0)
                self.masstransits.append(float(breakdown['masstransit'].strip('%'))/100.0)
                self.ridehailings.append(float(breakdown['ridehailing'].strip('%'))/100.0)
            else:
                print('foobar')
            
        self.score = mean(self.scores)
        self.bikeshare = mean(self.bikeshares)
        self.carshare = mean(self.carshares)
        self.masstransit = mean(self.masstransits)
        self.ridehailing = mean(self.ridehailings)

In [8]:
## for testing purposes

# hex = hexagon((0,0,0))
# hex.random_points(5)
# hex.points
# hex.api_scrape()
# hex.score, hex.bikeshares, hex.carshare, hex.masstransit, hex.ridehailing

In [9]:


def xy_generator(center,grid,size):
    cx = grid[0] * 1.5 * size + center[1] # using lat-long convention for the city centers so x is the second value in center but not grid
    cy = size * sqrt(3) * ( grid[1] + grid[0]/2 ) + center[0]
    x=[]
    y=[]
    
    for n in range(6):
        angle=n*pi/3
        x.append(size*cos(angle)+cx)
        y.append(size*sin(angle)+cy)
    return (x,y)

xy_generator((39,-77),(0,0),0.025)

([-76.975, -76.9875, -77.0125, -77.025, -77.0125, -76.9875],
 [39.0,
  39.02165063509461,
  39.02165063509461,
  39.0,
  38.97834936490539,
  38.97834936490539])

In [19]:
from tqdm import tqdm
import pickle

class grid_axial:
    def __init__(self, width, size=0.0025, coordinates = (38.904978, -77.039658)):
        self.width=width
        self.size=size
        self.coordinates = coordinates
        self.hex_dataframe_creator()
        self.polygons =[]
    def set_crs(self, crs):
        pass
    def make_ring(self,n):
        ring = []
        for m in range(1,n+1):
            ring.append(((n,-m,-n+m),m))
            ring.append(((-m,-n+m,n),m))
            ring.append(((-n+m,n,-m),m))
            ring.append(((-n,m, n-m),m))
            ring.append(((m,n-m,-n),m))
            ring.append(((n-m,-n,m),m))
        return ring
    
    def fill_grid(self):
        hex_list = [((0,0,0),0)]
        for n in range(1,self.width+1):
            hex_list+=self.make_ring(n)
        return hex_list
        
    def y_generator(self, grid):
        center = self.coordinates
        size = self.size
        cy = size * sqrt(3) * ( grid[1] + grid[0]/2 ) + center[0]
        # using lat-long convention for the city centers so x is the second value in center but not grid
        y=[]
        for n in range(6):
            angle=n*pi/3
            y.append(size*sin(angle)+cy)
        return y
    
    def x_generator(self,grid):
        center = self.coordinates
        size = self.size
        cx = 1.5 * float(grid[0]) * size + center[1] 
        # using lat-long convention for the city centers so x is the second value in center but not grid
        x=[]

        for n in range(6):
            angle=n*pi/3
            x.append(size*cos(angle)+cx)
        return x
    
    def hex_dataframe_creator(self):
        self.hexes = gpd.GeoDataFrame(self.fill_grid(), columns = ['hexagon','radius'])
        self.hexes['y'] = self.hexes.hexagon.apply(self.y_generator)
        self.hexes['x'] = self.hexes.hexagon.apply(self.x_generator)
    
    
    def poly_wag(self):        
        self.hexes['transit_score'] = [poly.score for poly in self.polygons]
        self.hexes['bike_score']  = [poly.bikeshare for poly in self.polygons]
        self.hexes['carshare_score']  = [poly.carshares for poly in self.polygons]
        self.hexes['mass_transit_score']  = [poly.masstransits for poly in self.polygons]
        self.hexes['ride_hail_score'] = [poly.ridehailings for poly in self.polygons]
    
    def save_partial(self):
        with open('partial.pkl', 'wb') as f:
            pickle.dump(self.polygons, f)    
        
    def check_randoms(self,n):
        hex_list = self.hexes['hexagon'].values
        i = 0
        for next in tqdm(hex_list):
            hex = hexagon(coords=next, city_center=self.coordinates, size=self.size)
            hex.random_points(n)
            hex.api_scrape()
            self.polygons.append(hex)
            i+=1
            if i==100:
                i=0
                self.save_partial()
        


        
# atlanta = grid_axial(hex_width,hex_size, coordinates=(atlanta_y,atlanta_x))
# atlanta.hexes.head()

In [20]:
dc = grid_axial(hex_width, hex_size, coordinates=(washington_y, washington_x))
dc.hexes.head()

,hexagon,radius,y,x
0,"(0, 0, 0)",0,"[38.904978, 38.90671005080757, 38.906710050807...","[-77.03765800000001, -77.038658, -77.040658000..."
1,"(1, -1, 0)",1,"[38.90324594919243, 38.904978, 38.904978, 38.9...","[-77.03465800000001, -77.035658, -77.037658000..."
2,"(-1, 0, 1)",1,"[38.90324594919243, 38.904978, 38.904978, 38.9...","[-77.04065800000001, -77.041658, -77.043658000..."
3,"(0, 1, -1)",1,"[38.90844210161514, 38.910174152422705, 38.910...","[-77.03765800000001, -77.038658, -77.040658000..."
4,"(-1, 1, 0)",1,"[38.90671005080757, 38.90844210161514, 38.9084...","[-77.04065800000001, -77.041658, -77.043658000..."


In [23]:
dc.check_randoms(1)

  1%|▌                                                                             | 54/7651 [02:15<5:17:17,  2.51s/it]

foobar


c:\programdata\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\programdata\anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  5%|████▏                                                                        | 414/7651 [14:12<4:08:18,  2.06s/it]

foobar


 11%|████████▍                                                                    | 835/7651 [27:52<3:47:34,  2.00s/it]

...

 93%|██████████████████████████████████████████████████████████████████████▍     | 7097/7651 [3:36:32<16:54,  1.83s/it]

foobar


 94%|███████████████████████████████████████████████████████████████████████▌    | 7204/7651 [3:39:40<13:37,  1.83s/it]

foobar


100%|████████████████████████████████████████████████████████████████████████████| 7651/7651 [3:52:45<00:00,  1.83s/it]


In [27]:

dc.hexes.head()

df1 = dc.hexes

In [34]:
poly = dc.polygons

len(dc.polygons), df1.shape

(7751, (7651, 4))

In [41]:
poly_df = pd.DataFrame(poly, columns = ['geometry'])

In [50]:
poly_df['transit_score'] = poly_df['geometry'].apply(lambda x: x.scores)
poly_df['bike_score']  = poly_df['geometry'].apply(lambda x: x.bikeshares)
poly_df['carshare_score']  = poly_df['geometry'].apply(lambda x: x.carshares)
poly_df['mass_transit_score']  = poly_df['geometry'].apply(lambda x: x.masstransits)
poly_df['ride_hail_score'] = poly_df['geometry'].apply(lambda x: x.ridehailings)


In [45]:
poly_df['coordinates'] = poly_df.geometry.apply(lambda x: (x.x_loc, x.y_loc, x.z_loc))

In [53]:
poly_df.head()

,geometry,coordinates,transit_score,bike_score,carshare_score,mass_transit_score,ride_hail_score
0,<__main__.hexagon object at 0x000001BE4BA19DD8>,"(0.0, 0.0, 0.0)",[98],[0.11],[0.02],[0.74],[0.13]
1,<__main__.hexagon object at 0x000001BE4BA19DA0>,"(1.0, -1.0, 0.0)",[100],[0.12],[0.03],[0.73],[0.12]
2,<__main__.hexagon object at 0x000001BE4BA195F8>,"(-1.0, 0.0, 1.0)",[100],[0.12],[0.03],[0.73],[0.12]
3,<__main__.hexagon object at 0x000001BE4BA40908>,"(0.0, 1.0, -1.0)",[100],[0.09],[0.02],[0.79],[0.1]
4,<__main__.hexagon object at 0x000001BE4B49AE80>,"(-1.0, 1.0, 0.0)",[100],[0.08],[0.03],[0.77],[0.12]


In [59]:
poly_df['bike_score'] = poly_df['bike_score'].apply(lambda x: sum(x))
poly_df['carshare_score'] = poly_df['carshare_score'].apply(lambda x: sum(x))
poly_df['mass_transit_score'] = poly_df['mass_transit_score'].apply(lambda x: sum(x))
poly_df['ride_hail_score'] = poly_df['ride_hail_score'].apply(lambda x: sum(x))
poly_df['transit_score'] = poly_df['transit_score'].apply(lambda x: sum(x))

poly_df.head()

,geometry,coordinates,transit_score,bike_score,carshare_score,mass_transit_score,ride_hail_score
0,<__main__.hexagon object at 0x000001BE4BA19DD8>,"(0.0, 0.0, 0.0)",98,0.11,0.02,0.74,0.13
1,<__main__.hexagon object at 0x000001BE4BA19DA0>,"(1.0, -1.0, 0.0)",100,0.12,0.03,0.73,0.12
2,<__main__.hexagon object at 0x000001BE4BA195F8>,"(-1.0, 0.0, 1.0)",100,0.12,0.03,0.73,0.12
3,<__main__.hexagon object at 0x000001BE4BA40908>,"(0.0, 1.0, -1.0)",100,0.09,0.02,0.79,0.10
4,<__main__.hexagon object at 0x000001BE4B49AE80>,"(-1.0, 1.0, 0.0)",100,0.08,0.03,0.77,0.12


In [71]:
df2 = df1.merge(poly_df[['coordinates', 'transit_score', 'bike_score',
       'carshare_score', 'mass_transit_score',
         'ride_hail_score']].groupby('coordinates', as_index=False).mean(),
          left_on='hexagon', right_on='coordinates')

In [73]:
df2.to_pickle('washington_v2')

In [75]:
df2.transit_score.value_counts()

100.0    6845
99.0      462
98.0      255
97.0       41
0.0        34
99.5       11
98.5        2
50.0        1
Name: transit_score, dtype: int64

In [24]:
# atlanta.check_randoms(5) 
import pickle
with open('hext_list_dc.pkl', 'wb') as f:
    pickle.dump(dc, f)    

In [701]:
def hex_round(x,y):
    z = -x -y
    x_d, x = modf(x)
    y_d, y = modf(y)
    z_d, z = modf(z)
    
    if x_d>y_d and x_d>z_d:
        x = -y-z
    elif y_d>z_d:
        y = -x-z
    else:
        z=-x-y
        
    return (int(x),int(y),int(z))

In [702]:
ssq = sqrt(3)/3

def point_loc_to_hex_loc(x_h, y_h, size):
    x = (x_h * 2/3)/ size
    y = ((-x_h/3) + (ssq * y_h)) / size
    return hex_round(x,y)


In [703]:
def hex_center(x_row, y_row, size):
    x = size * 3/2 * x_row
    y = size * (3**0.5) * (y_row + x_row/2)
    return x, y

In [704]:

hex_assign = lambda p: point_loc_to_hex_loc(p.x-washington_x, p.y-washington_y, hex_size)


In [ ]:
df['hexagon'] = df['geometry'].apply(hex_assign)
